# Notebook de testing

In [14]:
# Librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path
from scipy.stats import ttest_ind

# sube de notebooks/ a la raíz del repo

sys.path.append(str(Path.cwd().parent))

import src.functions as fc

%matplotlib inline 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
df1 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt")
df2 = fc.load_data_merge("20260212_1519_df2_merged.csv")
df3 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt")

In [16]:
df2.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [21]:
# Drop

df3 = df3.dropna()

## Tasa de finalización por grupo

In [ ]:
# Filtro por grupo

test = df3[df3["Variation"]=="Test"]
control = df3[df3["Variation"]=="Control"]

control

,client_id,Variation
2,4033851,Control
4,9294070,Control
7,6651403,Control
9,2105948,Control
12,9814849,Control
...,...,...
50491,4364429,Control
50493,8730282,Control
50494,5305116,Control
50495,393005,Control


In [ ]:
# Filtramos grupo de control

df_control = df2[df2["client_id"].isin(control["client_id"])]
df_test = df2[df2["client_id"].isin(test["client_id"])]

# Tasa de finalizacion por grupo

tasa_finalizacion_control = fc.tasa_finalizacion(df_control)
tasa_finalizacion_test = fc.tasa_finalizacion(df_test)

tasa_finalizacion_control, tasa_finalizacion_test

(65.59, 69.29)

## TEST